In [1]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "Times New Roman"
import sys
import os
sys.path.append("../../../../")
from ADFWI.propagator  import *
from ADFWI.model       import *
from ADFWI.view        import *
from ADFWI.utils       import *
from ADFWI.survey      import *
from ADFWI.fwi         import *

project_path = "./examples/acoustic/02-misfit-functions-test/00-Ricker-Test/Figures"

In [ ]:
nt = 2000
dt = 0.003
f0 = 6
f1 = 3
tlist,ricker = wavelet(nt,dt,f0,amp0=1,t0=0.5)
ricker       = np.roll(ricker,330)
std_noise    = 2
ricker       = ricker + np.random.normal(0,np.std(ricker),ricker.shape[0])*std_noise

tlist,f1_ricker     = wavelet(nt,dt,f0,amp0=1,t0=0.5)
f1_ricker           = np.roll(f1_ricker,330)
f1_ricker = f1_ricker + np.random.normal(0,np.std(f1_ricker),f1_ricker.shape[0])*0.2
shift_ricker        = np.roll(f1_ricker,-100)

plt.figure(figsize=(8,4))
plt.plot(tlist,ricker       ,c='r',linestyle="-"    ,label="Ricker wavelet")
plt.plot(tlist,shift_ricker ,c='b',linestyle="--"   ,label="Shifted ricker wavelet")
plt.legend(fontsize=14)
plt.xlabel("Times (s)",fontsize=13)
plt.ylabel("Amplitude",fontsize=13)
plt.tick_params(labelsize=13)
plt.show()

In [3]:
def normalize(data):
    if isinstance(data,list):
        data = np.array(data)
    res = (data - data.min())/(data.max() - data.min())
    return res

In [4]:
x_shift = np.arange(-200,200,1)

In [ ]:
# L1
from ADFWI.fwi.misfit import Misfit_waveform_L1
loss_fn = Misfit_waveform_L1(dt=1)

ricker_tensor = numpy2tensor(ricker)
ricker_tensor = ricker_tensor.reshape(1,-1,1)
L1_loss = []

for i in x_shift:
    shift_ricker = np.roll(f1_ricker,i)
    shift_ricker_tensor = numpy2tensor(shift_ricker)
    shift_ricker_tensor = shift_ricker_tensor.reshape(1,-1,1)
    loss = loss_fn.forward(ricker_tensor,shift_ricker_tensor)
    L1_loss.append(loss.detach().numpy())


plt.figure(figsize = (10,6))
plt.plot(x_shift,normalize(L1_loss),c='gray',label="L1")
plt.legend(fontsize=14)
plt.xlabel("Shift Distance",fontsize=14)
plt.ylabel("Normalized Misfit",fontsize=14)
plt.tick_params(labelsize=14)
plt.show()

In [ ]:
# L2
from ADFWI.fwi.misfit import Misfit_waveform_L2
loss_fn = Misfit_waveform_L2(dt=1)

ricker_tensor = numpy2tensor(ricker)
ricker_tensor = ricker_tensor.reshape(1,-1,1)
L2_loss = []
for i in x_shift:
    shift_ricker = np.roll(f1_ricker,i)
    shift_ricker_tensor = numpy2tensor(shift_ricker)
    shift_ricker_tensor = shift_ricker_tensor.reshape(1,-1,1)
    loss = loss_fn.forward(ricker_tensor,shift_ricker_tensor)
    L2_loss.append(loss.detach().numpy())

plt.figure(figsize = (10,6))
plt.plot(x_shift,normalize(L1_loss),c='gray',label="L1")
plt.plot(x_shift,normalize(L2_loss),c='silver',label="L2")
plt.legend(fontsize=14)
plt.xlabel("Shift Distance",fontsize=14)
plt.ylabel("Normalized Misfit",fontsize=14)
plt.tick_params(labelsize=14)
plt.show()

In [ ]:
# SmoothL1
from ADFWI.fwi.misfit import Misfit_waveform_smoothL1
loss_fn = Misfit_waveform_smoothL1(dt=1)

ricker_tensor = numpy2tensor(ricker)
ricker_tensor = ricker_tensor.reshape(1,-1,1)
SmoothL1_loss = []
for i in x_shift:
    shift_ricker = np.roll(f1_ricker,i)
    shift_ricker_tensor = numpy2tensor(shift_ricker)
    shift_ricker_tensor = shift_ricker_tensor.reshape(1,-1,1)
    loss = loss_fn.forward(ricker_tensor,shift_ricker_tensor)
    SmoothL1_loss.append(loss.detach().numpy())

plt.figure(figsize = (10,6))
plt.plot(x_shift,normalize(L1_loss)         ,c='gray'   ,label="L1")
plt.plot(x_shift,normalize(L2_loss)         ,c='silver' ,label="L2")
plt.plot(x_shift,normalize(SmoothL1_loss)   ,c='k'      ,label="SmoothL1")
plt.legend(fontsize=14)
plt.xlabel("Shift Distance",fontsize=14)
plt.ylabel("Normalized Misfit",fontsize=14)
plt.tick_params(labelsize=14)
plt.show()

In [ ]:
# StudentT
from ADFWI.fwi.misfit import Misfit_waveform_studentT
loss_fn = Misfit_waveform_studentT(dt=1)

ricker_tensor = numpy2tensor(ricker)
ricker_tensor = ricker_tensor.reshape(1,-1,1)
StudentT_loss = []
for i in x_shift:
    shift_ricker = np.roll(f1_ricker,i)
    shift_ricker_tensor = numpy2tensor(shift_ricker)
    shift_ricker_tensor = shift_ricker_tensor.reshape(1,-1,1)
    loss = loss_fn.forward(ricker_tensor,shift_ricker_tensor)
    StudentT_loss.append(loss.detach().numpy())

plt.figure(figsize = (10,6))
plt.plot(x_shift,normalize(L1_loss)         ,c='gray'   ,label="L1")
plt.plot(x_shift,normalize(L2_loss)         ,c='silver' ,label="L2")
plt.plot(x_shift,normalize(SmoothL1_loss)   ,c='k'      ,label="SmoothL1")
plt.plot(x_shift,normalize(StudentT_loss)   ,c='#00BBF9',label="StudentT")
plt.legend(fontsize=14)
plt.xlabel("Shift Distance",fontsize=14)
plt.ylabel("Normalized Misfit",fontsize=14)
plt.tick_params(labelsize=14)
plt.show()

In [ ]:
# Global Correlation
from ADFWI.fwi.misfit import Misfit_global_correlation
loss_fn = Misfit_global_correlation(dt=1)

ricker_tensor = numpy2tensor(ricker)
ricker_tensor = ricker_tensor.reshape(1,-1,1)
GC_loss = []
for i in x_shift:
    shift_ricker = np.roll(ricker,i)
    shift_ricker_tensor = numpy2tensor(shift_ricker)
    shift_ricker_tensor = shift_ricker_tensor.reshape(1,-1,1)
    loss = loss_fn.forward(ricker_tensor,shift_ricker_tensor)
    GC_loss.append(loss.detach().numpy())

plt.figure(figsize = (10,6))
plt.plot(x_shift,normalize(L1_loss)         ,c='gray'   ,label="L1")
plt.plot(x_shift,normalize(L2_loss)         ,c='silver' ,label="L2")
plt.plot(x_shift,normalize(SmoothL1_loss)   ,c='k'      ,label="SmoothL1")
plt.plot(x_shift,normalize(StudentT_loss)   ,c='#00BBF9',label="StudentT")
plt.plot(x_shift,normalize(GC_loss)         ,c='#22A884',label="Global Correlation")

plt.legend(fontsize=14)
plt.xlabel("Shift Distance",fontsize=14)
plt.ylabel("Normalized Misfit",fontsize=14)
plt.tick_params(labelsize=14)
plt.show()

In [ ]:
# Envelope Correlation
from ADFWI.fwi.misfit import Misfit_envelope
loss_fn = Misfit_envelope(dt=1,p=1,instaneous_phase=False)

ricker_tensor = numpy2tensor(ricker)
ricker_tensor = ricker_tensor.reshape(1,-1,1)
Envelope_loss = []
for i in x_shift:
    shift_ricker = np.roll(f1_ricker,i)
    shift_ricker_tensor = numpy2tensor(shift_ricker)
    shift_ricker_tensor = shift_ricker_tensor.reshape(1,-1,1)
    loss = loss_fn.forward(ricker_tensor,shift_ricker_tensor)
    Envelope_loss.append(loss.detach().numpy())

plt.figure(figsize = (10,6))
plt.plot(x_shift,normalize(L1_loss)         ,c='gray'   ,label="L1")
plt.plot(x_shift,normalize(L2_loss)         ,c='silver' ,label="L2")
plt.plot(x_shift,normalize(SmoothL1_loss)   ,c='k'      ,label="SmoothL1")
plt.plot(x_shift,normalize(StudentT_loss)   ,c='#00BBF9',label="StudentT")
plt.plot(x_shift,normalize(GC_loss)         ,c='#22A884',label="Global Correlation")
plt.plot(x_shift,normalize(Envelope_loss)   ,c='#EB1C4A',label="Envelope")

plt.legend(fontsize=14)
plt.xlabel("Shift Distance",fontsize=14)
plt.ylabel("Normalized Misfit",fontsize=14)
plt.tick_params(labelsize=14)
plt.show()

In [ ]:
# Softdtw
from ADFWI.fwi.misfit import Misfit_sdtw
loss_fn = Misfit_sdtw(dt=0.5,gamma=2,sparse_sampling=3)

ricker_tensor = numpy2tensor(ricker).to("cuda:0")
ricker_tensor = ricker_tensor.reshape(1,-1,1)
Sdtw_loss = []
for i in x_shift:
    shift_ricker = np.roll(f1_ricker,i)
    shift_ricker_tensor = numpy2tensor(shift_ricker).to("cuda:0")
    shift_ricker_tensor = shift_ricker_tensor.reshape(1,-1,1)
    loss = loss_fn.forward(ricker_tensor[:,:,:],shift_ricker_tensor[:,:,:])
    Sdtw_loss.append(loss.cpu().detach().numpy())

plt.figure(figsize = (10,6))
plt.plot(x_shift,normalize(L1_loss)         ,c='gray'   ,label="L1")
plt.plot(x_shift,normalize(L2_loss)         ,c='silver' ,label="L2")
plt.plot(x_shift,normalize(SmoothL1_loss)   ,c='k'      ,label="SmoothL1")
plt.plot(x_shift,normalize(StudentT_loss)   ,c='#00BBF9',label="StudentT")
plt.plot(x_shift,normalize(GC_loss)         ,c='#22A884',label="Global Correlation")
plt.plot(x_shift,normalize(Envelope_loss)   ,c='#EB1C4A',label="Envelope")
plt.plot(x_shift,normalize(Sdtw_loss)       ,c='#1E90FF',label="Softdtw")

plt.legend(fontsize=14)
plt.xlabel("Shift Distance",fontsize=14)
plt.ylabel("Normalized Misfit",fontsize=14)
plt.tick_params(labelsize=14)
plt.show()

## Wasserstein对于数据采样非常敏感，不能随便降采样

In [ ]:
# Wasserstein
from ADFWI.fwi.misfit import Misfit_Wasserstein1,Misfit_wasserstein_1d,Misfit_wasserstein_sinkhorn
# loss_fn = Misfit_Wasserstein1(trans_type='linear',dt=1,theta=1)
# loss_fn = Misfit_wasserstein_1d(dt=dt,p=1)
loss_fn = Misfit_wasserstein_sinkhorn(dt=dt,sparse_sampling=1,p=1,blur=1e-2,scaling=0.92)

ricker_tensor = numpy2tensor(ricker).to("cuda:0")
ricker_tensor = ricker_tensor.reshape(1,-1,1)
Wasserstein1_loss = []
for i in x_shift:
    shift_ricker = np.roll(ricker,i)
    shift_ricker_tensor = numpy2tensor(shift_ricker).to("cuda:0")
    shift_ricker_tensor = shift_ricker_tensor.reshape(1,-1,1)
    loss = loss_fn.forward(ricker_tensor,shift_ricker_tensor)
    Wasserstein1_loss.append(loss.cpu().detach().numpy())

plt.figure(figsize = (10,6))
plt.plot(x_shift,normalize(L1_loss)         ,c='gray'   ,label="L1")
plt.plot(x_shift,normalize(L2_loss)         ,c='silver' ,label="L2")
plt.plot(x_shift,normalize(SmoothL1_loss)   ,c='k'      ,label="SmoothL1")
plt.plot(x_shift,normalize(StudentT_loss)   ,c='#00BBF9',label="StudentT")
plt.plot(x_shift,normalize(GC_loss)         ,c='#22A884',label="Global Correlation")
plt.plot(x_shift,normalize(Envelope_loss)   ,c='#EB1C4A',label="Envelope")
plt.plot(x_shift,normalize(Sdtw_loss)       ,c='#1E90FF',label="Softdtw")
plt.plot(x_shift,normalize(Wasserstein1_loss),c='#FF8C00',label="Wasserstein1")

plt.legend(fontsize=14)
plt.xlabel("Shift Distance",fontsize=14)
plt.ylabel("Normalized Misfit",fontsize=14)
plt.tick_params(labelsize=14)
plt.show()

## Figure For Article

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import gridspec

fig = plt.figure(figsize = (12,10))
gs  = gridspec.GridSpec(7,2,hspace=0.8)
ax0 = fig.add_subplot(gs[0:2,0])
ax1 = fig.add_subplot(gs[0:2,1])
ax2 = fig.add_subplot(gs[2:,:])

################################
nt = 2000
dt = 0.003
f0 = 6
f1 = 3
tlist,ricker = wavelet(nt,dt,f0,amp0=1,t0=0.5)
ricker       = np.roll(ricker,330)
std_noise    = 2
noise_ricker = ricker + np.random.normal(0,np.std(ricker),ricker.shape[0])*std_noise
shift_ricker = np.roll(noise_ricker,0)

ax0.plot(tlist,ricker       ,c='k',linestyle="-"    ,label="Ricker")
ax0.plot(tlist,shift_ricker ,c='r',linestyle="--"   ,label="Shifted Ricker")
ax0.grid()
ax0.legend(fontsize=12,loc='upper right')
ax0.set_xlabel("Times (s)",fontsize=13)
ax0.set_ylabel("Amplitude",fontsize=13)
ax0.tick_params(labelsize=13)

################################
nt = 2000
dt = 0.003
f0 = 6
f1 = 3
tlist,ricker = wavelet(nt,dt,f0,amp0=1,t0=0.5)
ricker       = np.roll(ricker,330)
std_noise    = 2
noise_ricker = ricker + np.random.normal(0,np.std(ricker),ricker.shape[0])*std_noise
shift_ricker = np.roll(noise_ricker,100)

ax1.plot(tlist,ricker       ,c='k',linestyle="-"    ,label="Ricker")
ax1.plot(tlist,shift_ricker ,c='r',linestyle="--"   ,label="Shifted Ricker")
ax1.grid()
ax1.legend(fontsize=12,loc='upper right')
ax1.set_xlabel("Times (s)",fontsize=13)
ax1.set_ylabel("Amplitude",fontsize=13)
ax1.tick_params(labelsize=13)

#################################

ax2.plot(x_shift*dt,normalize(L1_loss)          ,c='gray'   ,label="L1"                 ,linestyle='-')
ax2.plot(x_shift*dt,normalize(L2_loss)          ,c='silver' ,label="L2"                 ,linestyle='-.')
ax2.plot(x_shift*dt,normalize(SmoothL1_loss)    ,c='k'      ,label="SmoothL1"           ,linestyle='--')
ax2.plot(x_shift*dt,normalize(StudentT_loss)    ,c='#00BBF9',label="StudentT"           ,linestyle='-')
ax2.plot(x_shift*dt,normalize(GC_loss)          ,c='#22A884',label="Global Correlation" ,linestyle='-.')
ax2.plot(x_shift*dt,normalize(Envelope_loss)    ,c='#EB1C4A',label="Envelope"           ,linestyle='--')
ax2.plot(x_shift*dt,normalize(Sdtw_loss)        ,c='#1E90FF',label="Softdtw"            ,linestyle='-')
ax2.plot(x_shift*dt,normalize(Wasserstein1_loss),c='#FF8C00',label="Wasserstein1"       ,linestyle='-.')

fill_shift = np.arange(35,65)
ax2.fill_between(fill_shift*dt, 0, 1,color='green', alpha=0.1,transform=plt.gca().get_xaxis_transform())
ax2.fill_between(-fill_shift*dt, 0, 1,color='green', alpha=0.1,transform=plt.gca().get_xaxis_transform())

bbox_args = dict(boxstyle="round", fc="lightblue",edgecolor="lightblue")
ax2.annotate('Global minimum', 
             xy=(0, 0),xytext=(0.1, 0.07),
             fontsize=15,bbox=bbox_args,
             arrowprops=dict(
                            facecolor='lightblue', 
                            alpha=0.6,
                            arrowstyle='fancy',
                            # arrowstyle='-|>',
                            connectionstyle='arc3,rad=-0.4',#有多个参数可选
                            color='k',
                            ))
ax2.annotate('Local minimum', 
             xy=(0.15, 0.55),xytext=(0.2, 0.3),
             fontsize=15,bbox=bbox_args,
             arrowprops=dict(
                            facecolor='lightblue', 
                            alpha=0.6,
                            arrowstyle='fancy',
                            # arrowstyle='-|>',
                            connectionstyle='arc3,rad=-0.4',#有多个参数可选
                            color='k',
                            ))

ax2.legend(fontsize=13)
ax2.set_xlabel("Time shift (s)",fontsize=16)
ax2.set_ylabel("Normalized Misfit",fontsize=16)
ax2.tick_params(labelsize=16)
plt.savefig(os.path.join(project_path,"misfit_shift_noise.png"),dpi=300,bbox_inches="tight")
plt.show()